In [ ]:
import pandas as pd 
import numpy as np 

# Open the csv files

In [ ]:
orders=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\orders.csv")
order_full_detail=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\order_products__prior.csv")
train=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\order_products__train.csv")
departments=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\departments.csv")
aisles=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\aisles.csv")
products=pd.read_csv(r"C:\Users\Lenovo\Downloads\Grocery recommend data\products.csv")

# Look inside of the files

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
order_full_detail.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [ ]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [ ]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [ ]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
products.shape

(49688, 4)

In [ ]:
train.shape[0]+order_full_detail.shape[0]

33819106

# Merging the training and the prior dataset together for full dataset

In [ ]:
order_full_detail=pd.concat([train,order_full_detail],ignore_index=True)

In [ ]:
order_full_detail.shape

(33819106, 4)

In [ ]:
products=products.merge(aisles,on="aisle_id",how="left")
products=products.merge(departments,on="department_id",how="left")
products.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


# We will combine features from other dataset to this one

In [ ]:
dataframe=products.copy()

# Adding new features to the dataframe

remove the unneccessary order column

In [ ]:
order_full_detail.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [ ]:
order_full_detail.product_id.unique().sum()

1234416216

In [ ]:
products.product_id.sum()

1234473516

meaning there are some of the products and their details missing
- Products are listed but has not been sold

In [ ]:
order_full_detail.drop(columns=["add_to_cart_order"],inplace=True)

we want to find out how many times the product has been reordered while purchasing

In [ ]:
order_full_detail.groupby(by="product_id")["reordered"].sum().reset_index()

,product_id,reordered
0,1,1185
1,2,13
2,3,209
3,4,161
4,5,10
...,...,...
49680,49684,1
49681,49685,6
49682,49686,89
49683,49687,6


Lets see a sample

In [ ]:
order_full_detail[order_full_detail["product_id"]==49685]["reordered"].sum()

6

We want to find out how many times the product has been purchased

In [ ]:
order_full_detail.groupby(by="product_id")["order_id"].nunique().reset_index()

,product_id,order_id
0,1,1928
1,2,94
2,3,283
3,4,351
4,5,16
...,...,...
49680,49684,9
49681,49685,49
49682,49686,127
49683,49687,14


We can see  sample

In [ ]:
order_full_detail[order_full_detail["product_id"]==49684] # 9 times this product has been purchased

,order_id,product_id,reordered
6705549,561618,49684,0
9977484,907116,49684,0
14344982,1367981,49684,1
23142864,2295026,49684,0
25580779,2551882,49684,0
25880810,2583499,49684,0
28244457,2832300,49684,0
29200897,2933365,49684,0
32059556,3235517,49684,0


In [ ]:
dataframe.shape

(49688, 6)

In [ ]:
col1=order_full_detail.groupby(by="product_id")["reordered"].sum().reset_index()
col2=order_full_detail.groupby(by="product_id")["order_id"].nunique().reset_index()
print(col1.shape)
print(col2.shape)

(49685, 2)
(49685, 2)


In [ ]:
dataframe=dataframe.merge(col1,on="product_id",how="left")
dataframe=dataframe.merge(col2,on="product_id",how="left")

dataframe.head()

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,1185.0,1928.0
1,2,All-Seasons Salt,104,13,spices seasonings,pantry,13.0,94.0
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,209.0,283.0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen,161.0,351.0
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry,10.0,16.0


## Finding and Filling Missing rows

Left Join (how="left"): A left join ensures that all rows from the left DataFrame (products) are retained. If there's a matching row in the right DataFrame (reordered_sum), the corresponding values will be added. If there's no match, the result will have NaN values for columns from the right DataFrame.

In [ ]:
dataframe[dataframe.isna().any(axis=1)]

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id
3629,3630,Protein Granola Apple Crisp,57,14,granola,breakfast,NaN,NaN
7044,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13,spreads,pantry,NaN,NaN
46624,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7,refrigerated,beverages,NaN,NaN


In [ ]:
dataframe.fillna(0,inplace=True)

In [ ]:
dataframe[dataframe.isna().any(axis=1)]

,product_id,product_name,aisle_id,department_id,aisle,department,reordered,order_id


## Droping unimportant rows and columns

In [ ]:
old_columns=dataframe.columns.tolist()
old_columns

['product_id',
 'product_name',
 'aisle_id',
 'department_id',
 'aisle',
 'department',
 'reordered',
 'order_id']

In [ ]:
new_columns=['product_id','product_name','aisle_id','department_id','aisle','department','reorder_total','order_total']

In [ ]:
dataframe.columns=new_columns

**support** is a key metric used to measure the frequency of an item or a combination of items being purchased together.

#### **Support in Grocery EDA:**

Support is defined as the ratio of transactions that contain a particular item (or a combination of items) to the total number of transactions. Mathematically:

$$
\text{Support}(X) = \frac{\text{Number of transactions containing item X}}{\text{Total number of transactions}}
$$

#### Example:

* If your dataset has 100 transactions and an item (say "Milk") appears in 40 transactions, then:

$$
\text{Support(Milk)} = \frac{40}{100} = 0.4
$$

This means **40% of transactions contain "Milk"**.

* **High Support**: Items that appear frequently in transactions. For example, items like "Bread" or "Rice" might have high support.
* **Low Support**: Items that appear less often, like niche or luxury products.

In EDA, **support** helps identify popular items or combinations that are often purchased together, which can be used to suggest **item bundling** or **cross-selling** strategies.

In [ ]:
dataframe.dtypes

In [ ]:
sum_of_reorder=dataframe["reorder_total"].sum()
sum_of_order=dataframe["order_total"].sum()

def support_reorder(x):
    return x / sum_of_reorder

def support_order(x):
    return x / sum_of_order

In [ ]:
dataframe["reorder_support"]=dataframe["reorder_total"].apply(support_reorder)
dataframe["order_support"]=dataframe["order_total"].apply(support_order)

In [ ]:
dataframe.head(3)

,product_id,product_name,aisle_id,department_id,aisle,department,reorder_total,order_total,reorder_support,order_support
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,1185.0,1928.0,5.938254e-05,0.000057
1,2,All-Seasons Salt,104,13,spices seasonings,pantry,13.0,94.0,6.514540e-07,0.000003
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,209.0,283.0,1.047338e-05,0.000008


In [ ]:
dataframe.department_id.nunique()

21

In [ ]:
dataframe.isnull().sum()

product_id         0
product_name       0
aisle_id           0
department_id      0
aisle              0
department         0
reorder_total      0
order_total        0
reorder_support    0
order_support      0
average_order      0
std_order          0
dtype: int64

In [ ]:
dataframe.groupby(by="department_id").value_counts()

department_id  product_id  product_name                                                       aisle_id  aisle           department  reorder_total  order_total  reorder_support  order_support  average_order  std_order  
1              4           Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce  38        frozen meals    frozen      161.0          351.0        8.068008e-06     1.037875e-05   583.193911     1948.568424    1
               8           Cut Russet Potatoes Steam N' Mash                                  116       frozen produce  frozen      90.0           178.0        4.510066e-06     5.263297e-06   583.193911     1948.568424    1
               12          Chocolate Fudge Layer Cake                                         119       frozen dessert  frozen      132.0          259.0        6.614764e-06     7.658393e-06   583.193911     1948.568424    1
               18          Pizza for One Suprema  Frozen Pizza                                79        froze

In [ ]:
dataframe["average_order"]=dataframe.groupby("department_id")["order_total"].transform("mean")
dataframe["std_order"]=dataframe.groupby("department_id")["order_total"].transform("std")
dataframe.head()

,product_id,product_name,aisle_id,department_id,aisle,department,reorder_total,order_total,reorder_support,order_support,average_order,std_order
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,1185.0,1928.0,5.938254e-05,5.700919e-05,479.950830,1370.273861
1,2,All-Seasons Salt,104,13,spices seasonings,pantry,13.0,94.0,6.514540e-07,2.779494e-06,364.330478,1378.163620
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,209.0,283.0,1.047338e-05,8.368051e-06,642.422680,2717.859168
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen,161.0,351.0,8.068008e-06,1.037875e-05,583.193911,1948.568424
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry,10.0,16.0,5.011185e-07,4.731054e-07,364.330478,1378.163620


In [ ]:
dataframe.drop(columns=["product_name"],inplace=True)
dataframe.set_index("product_id",inplace=True)
dataframe.head(10)

## Encode and Standardization and Increasing Columns for Better Clusturing (Neighbour algorithm)

In [ ]:
dataframe.dtypes

In [ ]:
numaric_columns=dataframe.select_dtypes(include=["int64","float64"]).columns.tolist()
categorical_columns=dataframe.select_dtypes(include=["object"]).columns.tolist()
categorical_columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
onehot=OneHotEncoder(handle_unknown="ignore")

numaric_pipeline=Pipeline(
    [
        ("scaler",scaler)
    ]
)
catagorical_pipeline=Pipeline(
    [
        ("onehot",onehot)
    ]
)

transformer=ColumnTransformer(
    [
        ("numaric",numaric_pipeline,numaric_columns),
        ("onehot",catagorical_pipeline,categorical_columns),
    ]
)


In [ ]:
Matrix=transformer.fit_transform(dataframe)

Matrix

In [ ]:
print(Matrix[0])

# ML Algorithm Train

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_nearest=NearestNeighbors(n_neighbors=5)
n_nearest.fit(Matrix)

In [ ]:
dataframe.head()

In [ ]:
distance, suggestions=n_nearest.kneighbors(Matrix[0],n_neighbors=5)  # distance, row index = model.kneighbours()
suggestions

In [ ]:
for i in suggestions.ravel():
    print(f'''{products.at[i,"product_name"]}  ------- {products.at[i,"aisle"]}  --------- {products.at[i,"department"]}''')

# Function to predict and showing altogether

In [ ]:
items_list=products.copy()
items_list.set_index("product_id",inplace=True)

In [ ]:
items_list[items_list["product_name"]=="Chocolate Sandwich Cookies"]

In [ ]:
print(items_list[items_list["product_name"]=="Chocolate Sandwich Cookies"].index[0])

In [ ]:
def suggest_item(data):
    index=items_list[items_list["product_name"]==data].index[0]
    values=n_nearest.kneighbors(Matrix[index],n_neighbors=5,return_distance=False)

    for i in values.ravel():
        print(f'''{items_list.at[i,"product_name"]}  ------- {items_list.at[i,"aisle"]}  --------- {items_list.at[i,"department"]}''')
        

In [ ]:
products.head()

In [ ]:
suggest_item("Robust Golden Unsweetened Oolong Tea")

# Working on next item predictor

In [ ]:
order_full_detail.sort_values(by="order_id",inplace=True)
order_full_detail.head(10)

In [ ]:
dictionary={
    1:list()
}
dictionary[1].append(4)
dictionary[1].append(5)
dictionary

In [ ]:
order_full_detail.order_id.max()

In [ ]:
products[products["product_id"]==1]

In [ ]:
products[products["product_id"]==1]["aisle_id"].values[0]

In [ ]:
order_full_detail.head()

## Caution for running this

If this file has been run once, then skip the next part as they take so much time and load the modified csv and start working

In [ ]:
purchase_list_dictionary={}

for i in range(1,order_full_detail.order_id.max()+1):
    purchase_list_dictionary[i]=list()

for row in order_full_detail.values:
   purchase_list_dictionary[row[0]].append(row[1])

In [ ]:
order_id_column=purchase_list_dictionary.keys()
purchase_aisle_column=purchase_list_dictionary.values()

In [ ]:
converted_dic=pd.DataFrame()
converted_dic["order_id"]=order_id_column
converted_dic["purchase_list"]=purchase_aisle_column

In [ ]:
converted_dic.head()

In [ ]:
converted_dic["total_product"]=converted_dic["purchase_list"].apply(len)

In [ ]:
converted_dic[(converted_dic["total_product"]>=10) & (converted_dic["total_product"]<=20)].head()

In [ ]:
converted_dic.to_csv("test.csv")

In [ ]:
max_purchase=converted_dic.total_product.max()

In [ ]:
target_df=converted_dic[(converted_dic["total_product"]>=10) & (converted_dic["total_product"]<=20)].copy()
target_df.shape

In [ ]:
dataframe[dataframe.index==4]["department_id"]

In [ ]:
l1=target_df["purchase_list"].values[0]

In [ ]:
target_df.head()

In [ ]:

def collect_department(list_data):
    modified_list = []
    for i,value in enumerate(list_data):
        dept=dataframe[dataframe.index==value]["department_id"].values[0]
        modified_list.append(dept)
    return modified_list

target_df["purchase_dept"]=target_df["purchase_list"].apply(collect_department)

In [ ]:
target_df.to_csv("target_df.csv",index=False)

## Run from here

In [ ]:
import pandas as pd
target_df=pd.read_csv("target_df.csv")
target_df.head()

,order_id,purchase_list,total_product,purchase_dept
0,4,"[21351, 41276, 32645, 25146, 17616, 40285, 348...",13,"[14, 7, 7, 7, 19, 19, 19, 19, 14, 14, 11, 3, 14]"
1,9,"[14992, 14183, 3990, 432, 27366, 18362, 23288,...",15,"[4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4, 3, 1..."
2,10,"[3464, 25720, 26842, 49235, 27156, 34134, 2212...",15,"[4, 4, 12, 16, 15, 18, 18, 4, 4, 4, 4, 4, 4, 4..."
3,12,"[37215, 34335, 37886, 25213, 15221, 30597, 316...",15,"[7, 7, 16, 19, 16, 16, 7, 4, 15, 7, 19, 19, 1,..."
4,13,"[36086, 25952, 3800, 23020, 33198, 41290, 2578...",13,"[16, 19, 16, 7, 7, 7, 7, 7, 13, 7, 5, 13, 5]"


In [ ]:
import ast
target_df['purchase_dept'] = target_df['purchase_dept'].apply(lambda x: ast.literal_eval(x))
target_df.head()

,order_id,purchase_list,total_product,purchase_dept
0,4,"[21351, 41276, 32645, 25146, 17616, 40285, 348...",13,"[14, 7, 7, 7, 19, 19, 19, 19, 14, 14, 11, 3, 14]"
1,9,"[14992, 14183, 3990, 432, 27366, 18362, 23288,...",15,"[4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4, 3, 1..."
2,10,"[3464, 25720, 26842, 49235, 27156, 34134, 2212...",15,"[4, 4, 12, 16, 15, 18, 18, 4, 4, 4, 4, 4, 4, 4..."
3,12,"[37215, 34335, 37886, 25213, 15221, 30597, 316...",15,"[7, 7, 16, 19, 16, 16, 7, 4, 15, 7, 19, 19, 1,..."
4,13,"[36086, 25952, 3800, 23020, 33198, 41290, 2578...",13,"[16, 19, 16, 7, 7, 7, 7, 7, 13, 7, 5, 13, 5]"


In [ ]:
training_set=[]
for seq in target_df["purchase_dept"].values:
    for val in range(1,len(seq)):
        training_set.append(seq[:val+1])

In [ ]:
training_set[:50]

[[14, 7],
 [14, 7, 7],
 [14, 7, 7, 7],
 [14, 7, 7, 7, 19],
 [14, 7, 7, 7, 19, 19],
 [14, 7, 7, 7, 19, 19, 19],
 [14, 7, 7, 7, 19, 19, 19, 19],
 [14, 7, 7, 7, 19, 19, 19, 19, 14],
 [14, 7, 7, 7, 19, 19, 19, 19, 14, 14],
 [14, 7, 7, 7, 19, 19, 19, 19, 14, 14, 11],
 [14, 7, 7, 7, 19, 19, 19, 19, 14, 14, 11, 3],
 [14, 7, 7, 7, 19, 19, 19, 19, 14, 14, 11, 3, 14],
 [4, 14],
 [4, 14, 16],
 [4, 14, 16, 16],
 [4, 14, 16, 16, 7],
 [4, 14, 16, 16, 7, 3],
 [4, 14, 16, 16, 7, 3, 7],
 [4, 14, 16, 16, 7, 3, 7, 13],
 [4, 14, 16, 16, 7, 3, 7, 13, 19],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4, 3],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4, 3, 15],
 [4, 14, 16, 16, 7, 3, 7, 13, 19, 4, 4, 4, 3, 15, 16],
 [4, 4],
 [4, 4, 12],
 [4, 4, 12, 16],
 [4, 4, 12, 16, 15],
 [4, 4, 12, 16, 15, 18],
 [4, 4, 12, 16, 15, 18, 18],
 [4, 4, 12, 16, 15, 18, 18, 4],
 [4, 4, 12, 16, 15, 18, 18, 4, 4],
 [

In [ ]:
maxlen = max(len(seq) for seq in training_set)
maxlen

20

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_training_set=pad_sequences(training_set,maxlen=maxlen+1,padding="pre")

In [ ]:
padded_training_set[:5]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 14,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 14,  7,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 14,  7,  7,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        14,  7,  7,  7, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 14,
         7,  7,  7, 19, 19]])

In [ ]:
X_train = padded_training_set[:, :-1]  # All columns except the last one (input sequence)
y_train = padded_training_set[:, -1]   # The last column (next product to predict)

In [ ]:
import numpy as np
np.max(padded_training_set)

21

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=np.max(padded_training_set) + 1)  # +1 for zero-padding

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout 

model=Sequential(
    [
        Embedding(input_dim=np.max(padded_training_set) + 1, output_dim=64, input_length=maxlen),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(np.max(padded_training_set) + 1, activation='softmax')  # Output layer for multi-class classification
    ]
)
model.build(input_shape=(None, maxlen))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 64)         │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22)             │           726 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,438 (595.46 KB)

 Trainable params: 152,438 (595.46 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity()